# Gerating artificial Signal Peptides (SPs)

## 0 Introduction 
To avoid the combinatorial explosion that arises from the large number of possible amino acid sequences, an algorithm must be developed to narrow down the search space and identify the sequences that are most likely to function as signal peptides. This can be accomplished through a variety of computational methods, such as bioinformatics, machine learning, and statistical analysis.

One common approach is to use bioinformatics methods to analyze large sets of data on known signal peptides and identify patterns or features that are associated with signal peptide function. These features can then be used to predict the function of novel sequences.

Machine learning algorithms can also be used to predict signal peptides. These algorithms can be trained on large sets of data on known signal peptides, and can then be used to predict the function of novel sequences. Common machine learning algorithms used for this purpose include decision trees, random forests, and neural networks.

Another approach is to use statistical analysis to identify the regions of the peptide sequences that are most likely to function as signal peptides. This can be done by analyzing the frequency and distribution of different amino acids in known signal peptides and identifying those that are over-represented or under-represented in these sequences.

In summary, by developing an algorithm, we can narrow down the search space and identify the sequences that are most likely to function as signal peptides, thus avoiding combinatorial explosion. The algorithm we are showcasing here is based on a combination of bioinformatics, machine learning and statistical analysis.

### Agenda
- Use AutoML predictions and synthetic signal peptide generation algorotihm to get a novel list of potential signal peptides

In [1]:
import numpy as np
import pandas as pd
import random
import os

In [2]:
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=22) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str


In [4]:
def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
    
    
    return sequence        

In [5]:
def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df


In [6]:
def split_peptides_sequences(df_signalPP:pd.DataFrame): 
    '''Split each AA for each position'''
    peptides_split = []
    for k,v in df_signalPP.iterrows(): 
        sequence = []
        for seq in v['sequence']: 
            sequence.append(seq)
        peptides_split.append(sequence)
    
    # make a dataframe
    new_peptides = pd.DataFrame(peptides_split)
    new_peptides = new_peptides.fillna('-')

    return new_peptides

## 1 Amino acid probability matrix

Lets import our df_pwn that was made in a previous notebook:

In [7]:
from google.colab import drive 
drive.mount('/content/home')

Mounted at /content/home


In [9]:
df_pwn =  pd.read_csv('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/02_All_signal_peptides/df_pwn_68_positions.csv')
df_pwn

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,-
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.069877,0.001889,0.002833,0.001889,0.032106,0.021719,0.064212,0.037771,0.208687,0.084986,...,0.022663,0.028329,0.058546,0.193579,0.033994,0.018886,0.052880,0.014164,0.028329,0.000000
2,0.044381,0.007554,0.006610,0.007554,0.152975,0.047214,0.019830,0.050047,0.018886,0.190746,...,0.020774,0.052880,0.020774,0.053824,0.131256,0.053824,0.060434,0.025496,0.022663,0.000000
3,0.057602,0.001889,0.004721,0.004721,0.089707,0.019830,0.016053,0.064212,0.058546,0.175637,...,0.033994,0.041549,0.043437,0.056657,0.163362,0.100094,0.021719,0.010387,0.026440,0.000000
4,0.072710,0.004721,0.004721,0.007554,0.054769,0.032106,0.019830,0.077432,0.053824,0.155807,...,0.041549,0.040604,0.036827,0.045326,0.138810,0.096317,0.043437,0.023607,0.032106,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.001889,0.000944,0.000000,0.000000,0.000000,0.997167
59,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,0.000000,0.000000,0.998111
60,0.000944,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000944,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.998111
61,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [10]:
amino_acids = list(df_pwn.columns.values)

In [11]:
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())

## 2 Load best model from AutoML

In [12]:
%%capture 
!pip install h2o

In [13]:
import h2o
from h2o.automl import H2OAutoML

In [14]:
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu220.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu220.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp4mavde1t
  JVM stdout: /tmp/tmp4mavde1t/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp4mavde1t/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,07 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_0t2zg0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [15]:
best_model = h2o.load_model("/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/Data/06_H2O_AutoML/06.1_H2O_AutoML_output/m30_DRF_1_AutoML_1_20230220_170537")
best_model

Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_1_AutoML_1_20230220_170537


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    37                 37                          88739                  19           20           19.973        136           219           184.622

ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.0016584398069097528
RMSE: 0.040723946357269364
MAE: 0.005297624901332574
RMSLE: 0.0315349957378654
Mean Residual Deviance: 0.0016584398069097528

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 0.001720611709543464
RMSE: 0.0414802568644827
MAE: 0.0054603678907358425
RMSLE: 0.03209598415116956
Mean Residual Deviance: 0.001720611709543464

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.00540665  0.00268409  0.0113935     0.00598967    0.00544716    0.00390626    0.00779467    0.00337831    0.00297274    0.00623934    0.00235715    0.00458773
mean_residual_deviance  0.00169696  0.00273163  0.00921778    0.00166671    0.00112677    0.000729293   0.00205229    0.000389167   0.000153953   0.00141026    5.96656e-05   0.000163708
mse                     0.00169696  0.00273163  0.00921778    0.00166671    0.00112677    0.000729293   0.00205229    0.000389167   0.000153953   0.00141026    5.96656e-05   0.000163708
r2                      -16.1891    48.6024     0.00395401    -0.0189738    -0.0256788    -0.0132167    -3.52815      -0.0614242    -0.106691     -154.452      -0.114402     -3.57443
residual_deviance       0.00169696  0.00273163  0.00921778    0.00166671    0.00112677    0.000729293   0.00205229    0.000389167   0.000153953   0.00141026    5.96656e-05   0.000163708
rmse                    0.0332918   0.0255739   0.0960093     0.0408254     0.0335674     0.0270054     0.0453023     0.0197273     0.0124078     0.0375535     0.00772435    0.0127948
rmsle                   0.0273572   0.0172572   0.0664092     0.0342043     0.0290593     0.0238763     0.038772      0.018088      0.0117974     0.0314702     0.00750576    0.0123892

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------
    2023-02-20 17:06:11  12.673 sec  0                  nan              nan             nan
    2023-02-20 17:06:11  12.825 sec  5                  0.0439875        0.00548191      0.0019349
    2023-02-20 17:06:11  12.945 sec  10                 0.0392728        0.00487661      0.00154236
    2023-02-20 17:06:11  13.067 sec  15                 0.0413963        0.00521893      0.00171365
    2023-02-20 17:06:11  13.194 sec  20                 0.0428157        0.00547595      0.00183319
    2023-02-20 17:06:11  13.340 sec  25                 0.0424898        0.00553433      0.00180539
    2023-02-20 17:06:11  13.464 sec  30                 0.0410843        0.00533753      0.00168792
    2023-02-20 17:06:11  13.581 sec  35                 0.0406527        0.00523584      0.00165264
    2023-02-20 17:06:11  13.629 sec  37                 0.0407239        0.00529762      0.00165844

Variable Importances: 
variable    relative_importance     scaled_importance       percentage
----------  ----------------------  ----

## 3 Signal peptide predictor algorithm

In [ ]:
def signal_peptide_predictor(list_of_probabilities, amino_acids, n_peptides,  number_of_iterations:int)-> pd.DataFrame:
    '''Predicts best signal peptides from a number of iterations'''

    data = pd.DataFrame()
    for i in range(0,number_of_iterations):
        new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, amino_acids, n_peptides=n_peptides, max_len = 30 )
        new_TO_NATURE_peptides = split_peptides_sequences(new_TO_NATURE_peptides)

        df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
        # make the df into categorical values
        for column in df_test.columns:
            if column != 'MM_N_peptide_abundance':
                df_test[column] = df_test[column].asfactor()

        #predict
        predicted = best_model.predict(df_test).as_data_frame()
        new_TO_NATURE_peptides['predictions'] = predicted['predict'].to_list()

        if len(data) == 0: 
            data = new_TO_NATURE_peptides.copy()
        else: 
            # concat to precious predictions
            data = pd.concat([data, new_TO_NATURE_peptides], axis=0)
            data = data.sort_values('predictions', ascending = False)
            data = data[0:100]   

        if i % 1000 == 0:
            print(f"Iteration {i}")
            
    return data

In [ ]:
%%time
lets_predict_signal_peptides = signal_peptide_predictor(list_of_probabilities, 
                                                        amino_acids,n_peptides =  1000,  
                                                        number_of_iterations = 2000)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Iteration 0
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.8/dist-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column '22' has levels not trained on: ["0"]
  warnings.warn(w)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

In [ ]:
lets_predict_signal_peptides

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,predictions
963,M,K,F,A,L,W,V,S,L,A,...,-,-,-,-,-,-,-,-,-,0.119850
502,M,K,F,A,T,W,V,S,Y,L,...,-,-,-,-,-,-,-,-,-,0.119842
411,M,K,F,A,I,W,I,I,L,V,...,-,-,-,-,-,-,-,-,-,0.119505
833,M,E,G,A,V,S,G,L,I,L,...,-,-,-,-,-,-,-,-,-,0.116878
375,M,H,S,A,A,W,K,F,L,V,...,-,-,-,-,-,-,-,-,-,0.115204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630,M,R,T,P,W,W,S,L,T,L,...,-,-,-,-,-,-,-,-,-,0.087434
274,M,M,L,T,V,W,C,V,F,G,...,-,-,-,-,-,-,-,-,-,0.087296
988,M,L,L,N,R,W,S,T,L,L,...,-,-,-,-,-,-,-,-,-,0.087214
986,M,F,L,A,L,S,T,L,A,L,...,-,-,-,-,-,-,-,-,-,0.087119


In [ ]:
lets_predict_signal_peptides.to_excel('/content/home/MyDrive/DTU-MASTER/DTU-Sem4/Thesis/sigpep/09-AutoML/lets_predict_signal_peptides.xlsx', index=False)
